## Environment
Using Kernel: `dan-dev-py312-r433`

-----

# Wang Lab Preprocess

#### Stage the data

- Staging Data from **source_data** to **derived_data**

**Files Derived:**
> - daf2d1_genes_*
> - n2d1_genes_*
> - rsks1d1_genes_*



-----
- **Helper functions**

In [1]:
# Get the Wormbase ID associated with the features in var_df

# The RAW DATA GSM7147953_N2D1/features.tsv.gz contains
# The features and there associated Wormbase_Ids

# The order aligns with the column names in x_df

# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE229022

import pandas as pd

def get_wormbase_ids_as_features(adata, features_file_path: str) -> pd.DataFrame:
    """
    Loads Wormbase IDs from a compressed features file and merges them into adata.var.

    """
    # Load features.tsv.gz with Wormbase IDs
    features_n2d1_df = pd.read_csv(
        features_file_path,
        sep="\t",
        header=None,
        compression='infer',
        names=['Wormbase_Id', 'features', 'Expression']
    ).set_index('features').drop(columns=['Expression'])

    # Copy adata.var and add position index
    features_df = adata.var.copy()
    features_df['position'] = range(len(features_df))

    # Merge to get Wormbase_Id aligned with adata.var index
    wormbase_ids_as_features = features_df.merge(
        features_n2d1_df,
        how='left',
        left_index=True,
        right_index=True
    )

    # Fill in missing Wormbase IDs with the index name
    wormbase_ids_as_features['Wormbase_Id'] = wormbase_ids_as_features['Wormbase_Id'].fillna(
        pd.Series(wormbase_ids_as_features.index, index=wormbase_ids_as_features.index)
    )

    # Sanity check (optional logging)
    print(wormbase_ids_as_features.head())
    print(f"{len(wormbase_ids_as_features):,}")
    print("All Wormbase_Id values are unique:", wormbase_ids_as_features['Wormbase_Id'].is_unique)

    return wormbase_ids_as_features

In [2]:
def extract_genotype_expression_df(
    genotype: str,
    wormbase_ids_as_features: pd.DataFrame,
    obs_df: pd.DataFrame,
    x_df: pd.DataFrame,
) -> pd.DataFrame:
    """
    Extract a gene expression DataFrame for a specific genotype from an AnnData object.
    """
    # Boolean mask for selected genotype
    genotype_mask = obs_df['genotype'] == genotype

    # Extract and label expression data
    x_genotype_df = x_df[genotype_mask.values].copy()
    
    # Apply Wormbase Ids to Columns
    x_genotype_df.columns = wormbase_ids_as_features['Wormbase_Id'].tolist()
    
    print(f"{x_genotype_df.shape=}")

    # Add tissue column from adata.obs
    x_genotype_df['tissue'] = obs_df.loc[genotype_mask, 'tissue'].values
    print(f"{x_genotype_df.shape=}")

    return x_genotype_df


In [3]:
# Method to create a gene_counts_tissue.csv 

# We expect to be able to get the same values as the Shiny App
# https://agingc.shinyapps.io/AgingC_expression/

import pandas as pd
from pathlib import Path

def generate_tissue_gene_counts(
    expression_df: pd.DataFrame,
    wormbase_ids_df: pd.DataFrame,
    output_path: str
) -> pd.DataFrame:
    """
    Generate a tissue-by-gene expression count table from a single-cell expression DataFrame.
    """

    # Get all gene columns
    gene_cols = [col for col in expression_df.columns if col != 'tissue']

    # Binarize expression data
    binary_expr = (expression_df[gene_cols] > 0).astype(int)

    # Add tissue info back
    binary_expr['tissue'] = expression_df['tissue']

    # Group by tissue and count non-zero expressions
    tissue_gene_counts = binary_expr.groupby('tissue').sum()

    # Transpose to have genes as rows, tissues as columns
    gene_counts_tissue = tissue_gene_counts.T

    # Merge with WormBase gene info
    gene_counts_tissue = gene_counts_tissue.merge(
        wormbase_ids_df,
        how='left',
        left_index=True,
        right_on='Wormbase_Id'
    )

    # Arrange columns
    cols = ['position', 'features', 'Wormbase_Id'] + \
           [col for col in gene_counts_tissue.columns if col not in ['position', 'features', 'Wormbase_Id']]
    gene_counts_tissue = gene_counts_tissue[cols]

    # Debug print
    print(f"Columns found: {len(gene_counts_tissue.columns)}")
    # print("Column Names:")
    # for col in cols:
    #     print(f"   {col}")

    # Save to CSV
    directory = Path(output_path).parent
    directory.mkdir(parents=True, exist_ok=True)
    gene_counts_tissue.to_csv(output_path, index=False)

    return gene_counts_tissue

In [4]:
# Method to create a tissue_gene_counts.csv for Day 1 WT Worms
# This will help us evaluate tissues at Day 1

# We expect to be able to get the same values as the Shiny App
# https://agingc.shinyapps.io/AgingC_expression/

import pandas as pd
from pathlib import Path

def generate_tissue_gene_expression(
    expression_df: pd.DataFrame,
    wormbase_ids_df: pd.DataFrame,
    output_path: str
) -> pd.DataFrame:
    """
    Generate a tissue-by-gene expression matrix from a single-cell gene expression DataFrame.
    """
    # Sum expression values grouped by tissue
    tissue_gene_expression = expression_df.groupby('tissue').sum()

    # Transpose to have genes as rows, tissues as columns
    gene_expression_tissue = tissue_gene_expression.T

    # Merge gene metadata (WormBase ID and features)
    gene_expression_tissue = gene_expression_tissue.merge(
        wormbase_ids_df,
        how='left',
        left_index=True,
        right_on='Wormbase_Id'
    )

    # Reorder columns: metadata first, then expression values
    metadata_cols = ['position', 'features', 'Wormbase_Id']
    expression_cols = [col for col in gene_expression_tissue.columns if col not in metadata_cols]
    ordered_cols = metadata_cols + expression_cols
    gene_expression_tissue = gene_expression_tissue[ordered_cols]

    # Output diagnostic info
    print(f"Columns found: {len(gene_expression_tissue.columns)}")
    # print("Column Names:")
    # for col in ordered_cols:
    #     print(f"   {col}")

    # Save to CSV
    directory = Path(output_path).parent
    directory.mkdir(parents=True, exist_ok=True)
    gene_expression_tissue.to_csv(output_path, index=False)

    return gene_expression_tissue

-----

- **Preprocess** (Get: gene_counts & expression_counts)

In [5]:
# Location of data from and to
from pathlib import Path

root_dir_path = Path("..")
source_data_path = root_dir_path / "source_data/wang_lab"
derived_data_path = root_dir_path / "derived_data/wang_lab" 

In [6]:
%%time
import pandas as pd
from scipy.sparse import csr_matrix

# The data was converted to h5ad from R seurat
import anndata as ad
print(f"anndata version {ad.__version__}")
adata = ad.read_h5ad(source_data_path / "seurat_obj.h5ad")

print(adata)

x_df = pd.DataFrame(data=csr_matrix.todense(adata.X))
#x_df


anndata version 0.11.4
AnnData object with n_obs × n_vars = 241969 × 26603
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'genotype', 'cb', 'singlelet', 'integrated_snn_res.0.5', 'integrated_snn_res.0.8', 'integrated_snn_res.1', 'integrated_snn_res.1.2', 'integrated_snn_res.1.5', 'seurat_clusters', 'tissue'
    var: 'features'
    obsm: 'X_tsne', 'X_umap'
CPU times: user 15.8 s, sys: 7.17 s, total: 22.9 s
Wall time: 23.1 s


In [7]:
wormbase_ids_as_features = get_wormbase_ids_as_features(adata, f"{source_data_path}/GSE229022_RAW/GSM7147953_N2D1_features.tsv.gz" )

genotypes = ['N2D1', 'DAF2D1', 'RSKS1D1']
for genotype in genotypes:
    x_genotype_df = extract_genotype_expression_df(genotype, wormbase_ids_as_features, obs_df=adata.obs, x_df=x_df)
    generate_tissue_gene_counts(x_genotype_df, wormbase_ids_as_features, f"{derived_data_path}/expression_data/{genotype.lower()}_gene_counts_tissue.csv")
    generate_tissue_gene_expression(x_genotype_df, wormbase_ids_as_features, f"{derived_data_path}/expression_data/{genotype.lower()}_gene_expression_tissue.csv")
    

       features  position     Wormbase_Id
nduo-6   nduo-6         0  WBGene00010957
ndfl-4   ndfl-4         1  WBGene00010958
MTCE.7   MTCE.7         2  WBGene00014454
nduo-1   nduo-1         3  WBGene00010959
atp-6     atp-6         4  WBGene00010960
26,603
All Wormbase_Id values are unique: True
x_genotype_df.shape=(22962, 26603)
x_genotype_df.shape=(22962, 26604)
Columns found: 20
Columns found: 20
x_genotype_df.shape=(13167, 26603)
x_genotype_df.shape=(13167, 26604)
Columns found: 20
Columns found: 20
x_genotype_df.shape=(12883, 26603)
x_genotype_df.shape=(12883, 26604)
Columns found: 20
Columns found: 20


-----
- **Extended Helper functions**

#### Functions to Create GT-ITF

### GF-ITF

__GF-ITF__ (Gene Frequency–Inverse Tissue Frequency) scores genes based on how specific they are to a tissue—highlighting genes that are highly expressed in one tissue but rare across others. It’s useful for identifying tissue-specific markers and filtering out broadly expressed housekeeping genes in enrichment analysis.

------

#### Gene Frequency (GF)
- Measures how frequently a gene appears in a specific tissue.

    - GF(g, t) = $ \frac{\text{Number of times gene } g \text{ appears in tissue } t}{\text{Total gene counts in tissue } t} $

#### Inverse Tissue Frequency (ITF)
- Penalizes genes that appear in many tissues.
    - $N$: Total number of tissues  
    - $n_g$: Number of tissues where gene $g$ is present  

    - ITF(g) = $ \log\left(1 + \frac{N}{1 + n_g}\right) $

#### GF-ITF
- The final GF-ITF score combines specificity and frequency.
    - Genes frequent in a tissue but rare across all tissues get a __high score__.
    - Housekeeping genes get a __low score__.

- GF-ITF(g, t) = $ \text{GF}(g, t) \times \text{ITF}(g) $

In [8]:
import numpy as np
import pandas as pd

def compute_gf_itf(expression_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute GF-ITF values for gene expression data.

    Parameters:
    - expression_df (pd.DataFrame): Gene expression matrix (genes x tissues)

    Returns:
    - pd.DataFrame: GF-ITF matrix (genes x tissues)
    """
    # GF: gene count / total gene counts per tissue
    gf = expression_df.div(expression_df.sum(axis=0), axis=1)

    presence = (expression_df > 0).astype(int)

    # ITF: log(1+ (total tissues / (1 + number of tissues where gene is present)))
    num_tissues = expression_df.shape[1]
    gene_presence_counts = presence.sum(axis=1)
    itf = np.log(1 + (num_tissues / (1 + gene_presence_counts)))

    # GF-ITF
    gf_itf = gf.multiply(itf, axis=0)

    return gf_itf

In [9]:
import pandas as pd
import numpy as np

def compute_gf_itf_with_single_cell_data(expression_df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes GF-ITF values for a gene expression matrix, preserving and restoring
    'position' and 'features' columns as the first columns after the index.

    """
    # Preserve metadata columns
    metadata = expression_df[['position', 'features']].copy() if {'position', 'features'}.issubset(expression_df.columns) \
        else expression_df[[col for col in ['position', 'features'] if col in expression_df.columns]].copy()

    # Drop metadata columns from expression data
    expr_only_df = expression_df.drop(columns=metadata.columns, errors='ignore')

    # Compute GF-ITF
    gf_itf_df = compute_gf_itf(expr_only_df)

    # Combine metadata and GF-ITF, with metadata as the first columns
    result_df = pd.concat([metadata, gf_itf_df], axis=1)

    return result_df

-----

- **Preprocess** (Get: gene_gf_tif_counts)

In [10]:
genotypes = ['N2D1', 'DAF2D1', 'RSKS1D1']
for genotype in genotypes:
    gene_counts_tissue_df =pd.read_csv(f"{derived_data_path}/expression_data/{genotype.lower()}_gene_counts_tissue.csv", index_col="Wormbase_Id")
    gf_itf_df = compute_gf_itf_with_single_cell_data(gene_counts_tissue_df)
    gf_itf_df.to_csv(f"{derived_data_path}/expression_data/{genotype.lower()}_gene_gf-itf_tissue.csv", index=True, index_label="Wormbase_Id")



-----
- **Extended Helper functions**

In [11]:
import pandas as pd
from pathlib import Path

def read_single_cell_data(genotype:str, data_type:str, drop_columns=True):
    """Reads and formats single-cell gene data by genotype and data type (e.g., 'counts' or 'expression')."""
    file_path = f"{derived_data_path}/expression_data/{genotype.lower()}_gene_{data_type}_tissue.csv"
 
    print(f"processing {file_path}")
    gene_tissue_df = pd.read_csv(file_path)
    gene_tissue_df = gene_tissue_df.set_index('Wormbase_Id')
    if drop_columns:
        gene_tissue_df = gene_tissue_df.drop(['position','features'], axis=1)
    return gene_tissue_df

def create_single_cell_data_dict(gene_tissue_df, gene_set_size):
    """Returns a dictionary mapping each tissue to its top expressed genes up to a specified set size."""
    gene_tissue_dict = {}

    for tissue in gene_tissue_df.columns:
        nonzero_genes = gene_tissue_df[gene_tissue_df[tissue] > 0]
        sorted_genes = (
            nonzero_genes
            .reset_index()
            .sort_values(by=[tissue, 'Wormbase_Id'], ascending=[False, True])
            .set_index('Wormbase_Id')
        )
        top_wormbase_ids = sorted_genes.head(gene_set_size).index.tolist()
        gene_tissue_dict[tissue] = top_wormbase_ids
    return gene_tissue_dict

def write_single_cell_data(gene_tissue_dict, genotype:str, data_type:str, gene_set_size:int):
    """Writes a dictionary of tissue-specific gene lists to an Excel file with one sheet per tissue."""
    batch_excel_path = f"{derived_data_path}/{gene_set_size}/{genotype.lower()}_gene_{data_type}_tissue_{gene_set_size}.xlsx"
    directory = Path(batch_excel_path).parent
    directory.mkdir(parents=True, exist_ok=True)
    # Save dictionary of lists to an Excel file with each key as a sheet
    with pd.ExcelWriter(batch_excel_path) as writer:
        for tissue, wormbase_ids in gene_tissue_dict.items():
            df = pd.DataFrame(wormbase_ids, columns=["Wormbase_Id"])
            df.to_excel(writer, sheet_name=str(tissue)[:31], index=False)
            

-----

- **Preprocess** (Get: limiting counts 500, 1000)

In [12]:
genotypes = ['N2D1', 'DAF2D1', 'RSKS1D1']
data_types = ['counts', 'expression', 'gf-itf']
gene_set_sizes = [500, 1000]
for genotype in genotypes:
    for data_type in data_types:
        single_cell_data_df = read_single_cell_data(genotype, data_type)
        for gene_set_size in gene_set_sizes:
            single_cell_data_dict = create_single_cell_data_dict(single_cell_data_df, gene_set_size)
            write_single_cell_data(single_cell_data_dict, genotype, data_type, gene_set_size)

processing ../derived_data/wang_lab/expression_data/n2d1_gene_counts_tissue.csv
processing ../derived_data/wang_lab/expression_data/n2d1_gene_expression_tissue.csv
processing ../derived_data/wang_lab/expression_data/n2d1_gene_gf-itf_tissue.csv
processing ../derived_data/wang_lab/expression_data/daf2d1_gene_counts_tissue.csv
processing ../derived_data/wang_lab/expression_data/daf2d1_gene_expression_tissue.csv
processing ../derived_data/wang_lab/expression_data/daf2d1_gene_gf-itf_tissue.csv
processing ../derived_data/wang_lab/expression_data/rsks1d1_gene_counts_tissue.csv
processing ../derived_data/wang_lab/expression_data/rsks1d1_gene_expression_tissue.csv
processing ../derived_data/wang_lab/expression_data/rsks1d1_gene_gf-itf_tissue.csv
